In [ ]:
#pip install pony

In [1]:
# импорт необходимых библеотек
from pony.orm import *
import getpass
import psycopg2
from datetime import date
#from models import db

In [2]:
# безопасно запрашиваем ввод пароля в CLI
password = getpass.getpass(prompt="Введите секретный пароль:")

Введите секретный пароль:········


# Создание базы данных Sentiment

In [3]:
# создание подключения к базе данных 'postgres'
connect = psycopg2.connect(database='postgres',
                       user='postgres',
                       password=password,
                       host='127.0.0.1',
                       port='5432')

print('База данных успешно открыта')

База данных успешно открыта


In [4]:
# задание 'autocommit'
connect.autocommit = True

In [5]:
# создание объекта курсора подключения к базе данных 'postgres'
cursor = connect.cursor()

In [7]:
# создание базы данных 'sentiment'
cursor.execute("CREATE database sentiment")

In [6]:
# закрытие подключения к базе данных 'postgres'
connect.close()

# Cоздание модели данных Sentiment

In [3]:
# создание объекта подключения к базе данных
db = Database()

In [4]:
# подключение к базе данных 'sentiment' PostgreSQL
db.bind(provider='postgres', user='postgres', password=password,
        host='127.0.0.1', database='sentiment', port='5432')

[Модель данных](https://editor.ponyorm.com/user/RusLat/sentiment/designer)

<img src='images/sentiment.png'>

In [5]:
# модель данных
class Sentiment(db.Entity):
    """комментарии (отзывы) пользователей"""
    _resource_ = 'https://www.imdb.com/'
    sentiment_id = PrimaryKey(int, column='sentiment_id', auto=True)  # идентификатор комментария (отзыва)
    rating = Optional(int, column='rating')  # рейтинг комментария (отзыва)
    review = Required(str, column='review')  # комментарий (отзыв) пользователя
    label_id = Required('Label')  # идентификатор типа комментария (отзыва)
    data_id = Required('Data', column='data_id')  # идентификатор выборки
    title_id = Required('Title', column='title_id')  # идентификатор url фильма


class Title(db.Entity):
    """url фильмов"""
    _resource_ = 'https://www.imdb.com/'
    title_id = Set(Sentiment)  # идентификатор url фильма
    title_url = Required(str, unique=True, column='title_url')  # url фильма


class Data(db.Entity):
    """тип выборки"""
    _resource_ = 'https://www.imdb.com/'
    data_id = Set(Sentiment)  # идентификатор выборки
    data_name = Required(str, unique=True, column='data_name')  # наименование выборки


class Label(db.Entity):
    """тип комментариев (отзывов)"""
    _resource_ = 'https://www.imdb.com/'
    label_id = Set(Sentiment)  # идентификатор типа комментария (отзыва)
    label_name = Required(str, unique=True, column='label_name')  # тип комментария (отзыва)


class Sentiment_New(db.Entity):
    """комментарии (отзывы) пользователей"""
    sentiment_id = PrimaryKey(int, column='sentiment_id', auto=True)  # идентификатор комментария (отзыва)
    date = Required(date, column='date')  # дата комментария (отзыва)
    review = Required(str, column='review')  # комментарий (отзыв) пользователя
    rating_pred = Optional(int, column='rating_pred')  # предсказанный рейтинг комментария (отзыва)

In [6]:
#db.generate_mapping()
#db.drop_all_tables(with_all_data=True)

In [7]:
# создание таблиц
db.generate_mapping(create_tables=True)